**<h1 style="text-align: center;">Stroke Prediction Project</h1>**

## 👋🏼 **Introduction**

<p style="text-align: justify;">I am a data scientist for a company called TrackFit. And it is building an application which tracks the health data for the user. And they are planning to add an additonal function where it will take some inputs and track the body mass index and based on these features it will try to predict whether user has chances of getting a stroke or not. Strokes are one of the many comman cause of deaths. The aim of our company is reduce the number of deaths that are caused due to heart stroke. But giving this stroke prediction function to the user, we are make the user beware of the potential harm and inform him/her to get a doctors opinion at the earliest, which will result is reducing the cause of deaths.</p>


## 🧿 **Objective**
 <p style="text-align: justify;">Then objective of this project is to build an machine learning module which will predict whether an user has chances of getting a hear stroke. For this my company are given me a dataset which has many metrics which has impact on the cause of stroke. So I will be using these features and building an accurate machine learning model. I have specified the steps we will be following in the **Table of content** so that it will be easier to navigate.
 
 </p>

**About the data**  
This dataset contains 12 columns. This data set is from Kaggle. You can [**click here**](https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset) to view the dataset on kaggle.

## **Table of Content**

##### ⬇ [**Importing Libraries**](#library)
##### 📊 [**Importing Dataset**](#dataset)
##### 🗺️ [**Exploring the dataset**](#exploration)
##### 💭 [**My taughts on the dataset**](#thoughts)
##### ⚙️ [**Pre-processing the data**](#processing)
##### 🔧 [**Feature Engineering**](#feature)
##### 🗂️ [**Model Selection**](#selection)
##### 🤖 [**Model Training**](#model)
##### ✅ [**Model Assessment**](#assess)
##### 🎛️ [**Hyper-Parameter Tuning**](#tuning)
##### 🏁 [**Final Model**](#Final)
##### 💡 [**Conclusion**](#conclusion)


## ***Let's begin....***

<a id='library'></a>
## ⬇ <span style="color: #20479b; font-weight: bold;">Importing Libraries</span>

I start by importing the necessary libraries for my Exploratory Data Analysis tasks in Python.

In [ ]:
import   pandas as  pd
import warnings
import    numpy as  np
import sklearn.model_selection
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from    sklearn.preprocessing import   OneHotEncoder
from   sklearn.preprocessing import StandardScaler
from    sklearn.compose import  ColumnTransformer
from  imblearn.over_sampling import SMOTE
from  sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm  import SVC
from sklearn.ensemble   import RandomForestClassifier
from sklearn.tree  import   DecisionTreeClassifier   
import sklearn.metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from  sklearn.metrics import roc_auc_score
from  sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
warnings.filterwarnings('ignore')

<a id='dataset'></a>  

## 📊 <span style="color: #20479b; font-weight: bold;">Importing Dataset</span>

In [ ]:
dataset = pd.read_csv('/Users/jasonjoelpinto/Documents/GitHub/python-datascience-projects/010. stroke_prediction/dataset/healthcare-dataset-stroke-data.csv')

print("No. of rows.   :", dataset.shape[0])
print("No. of cols.   :", dataset.shape[1])  
print("="*30)

dataset.rename( columns = {
                                'id':'identity',
                                'gender':'sex',
                                'hypertension': 'high_blood_pressure',
                                'heart_disease':'cardiovascular_disease' ,
                                'ever_married':'marrital_status' ,   
                                'work_type':'profession' ,
                                'Residence_type':'stay',
                                'avg_glucose_level':'blood_sugar_level' ,
                                'bmi':'body_mass_index' ,
                                'smoking_status':'tobaco_use' }
                                ,inplace=True)

dataset.sample(1)

Here I have renamed the column for better visiblity and understanding. and then I wwe can see a 

<a id='exploration'></a>
## 🗺️ <span style="color: #20479b; font-weight: bold;">Exploring the dataset</span>

**Note:**
For EDA and Data Pre-processing I will be using the main datafram. Once dataframe is cleaned. I will be splitting it into `train` and  `test` split. Here's why
1. I do not want the just the `test` data to be cleaned and `train` data having unwanted data and error.
2. If I clean only the `test` data then my data is not generalized. Hence while my model tries to predict it will have different issues during training and testing.

After loading the dataset into the DataFrame `dataset`, I will check the `dataset` shape using the `.shape` attribute.

In [ ]:
dataset.shape

As you can see above there are 5110 rows and 12 columns

Now I am going to check how much percentage of the columns are null. If they are negligible percentage then we can just drop the columns. Otherwise I will fill the na values with the meaninfull values.

In [ ]:
null_percentage = {}
for column in dataset:
    null_percentage[column] = round(( ( dataset[column].isnull().sum() ) / len(dataset)) * 100,2)
np_df = pd.DataFrame( list( null_percentage.items() ),  columns=[  'column_name'  , 'null_percentage' ])
np_df

As the null percentage of column `body_mass_index` is negligible when compared to the size of the dataset, we can drop these null rows as body mass index cannot be `NULL`.

Using `.duplicated().any()` I will be checking if there are any duplicate entries.

In [ ]:
dataset.duplicated().any()

There are no duplicates.

In [ ]:
sns.histplot(dataset, x='age')

Good to see data for all age groups.

Now let's check why there was data for age less than 1 year old's.

In [ ]:
print(dataset[dataset['age']<1].shape)
print('='*30)

There are 42 rows, and the body mass index looks less. There are chances that these are records for the infants. Let's check if all the records `worktype` is children.

In [ ]:
print(dataset[dataset['age']<1].equals(dataset[(dataset['age']<1) & (dataset['profession'] == 'children')]))
print('='*30)
dataset.loc[dataset['age']<1,['body_mass_index']].mean()

As you can both the dataframes match and also teh body mass index ais less is very less. Hence we I think that records are for infants. So we need not delete these.

In [ ]:
fig, (p1,p2,p3) = plt.subplots(1,3, figsize=(13, 6), dpi=300)

propotion = dataset['high_blood_pressure'].value_counts()
names = ['No','Yes']
p1.pie(propotion, labels=names, autopct='%1.1f%%')
p1.set_title('high_blood_pressure', fontweight='bold')

propotion = dataset['cardiovascular_disease'].value_counts()
names = ['No','Yes']
p2.pie(propotion, labels=names, autopct='%1.1f%%')
p2.set_title('Heart Disease', fontweight='bold')

propotion = dataset['marrital_status'].value_counts()
names = ['No','Yes']
p3.pie(propotion, labels=names, autopct='%1.1f%%')
p3.set_title('Married', fontweight='bold')

In this above plot you can see that
1. **high_blood_pressure:** Only 9.2% of all the people have high_blood_pressure problem.
2. **Heart Disease:** About 95.0% of all the people do not have Heart Disease problem.
3. **Married:** About 34.7% are married.


In [ ]:
dataset['stroke'].value_counts().plot(kind='pie')
plt.title('Proportion for stroke column', fontweight='bold')

Here you can see that the `data is imbalanced`, specially since we are focussing on stroke detection we have very less data for people you got stroke.

In [ ]:
fig, (b1, b2) = plt.subplots(1, 2, figsize=(8, 4), dpi=300)

sns.histplot(data=dataset, x="blood_sugar_level", ax=b1)
sns.histplot(data=dataset, x="body_mass_index", ax=b2)
plt.subplots_adjust(left=0.1, right=0.8, top=0.8, bottom=0.1, wspace=0.4)

plt.show()

In [ ]:
avg_of_numerical_cols = dataset.groupby('stroke')[['body_mass_index', 'blood_sugar_level', 'age']].mean().reset_index()
names = ['No Stroke',  'Stroke']
colors = ['#82b4a2',   '#dd987b']
barwidth = 0.4

fig, p = plt.subplots(1, 3, figsize=(13, 6), dpi=400)

for i, col in enumerate(['body_mass_index', 'blood_sugar_level', 'age']):
    x = avg_of_numerical_cols['stroke']
    y = avg_of_numerical_cols[col]


    p[i].bar(x,  y, label=names,  color =  colors, width =  barwidth, capstyle='round')
    p[i].set_title(f'Average {col} for data\n with stroke and without.', fontweight ='bold', pad  =20)
    p[i].set_ylabel(col , fontweight='bold')
    p[i].set_xticks(x)
    p[i].set_xticklabels(names, fontweight='bold')
    p[i].legend(title='Color'  )


plt.subplots_adjust(left=0.0, right=2.4, top=1, bottom=0.1, wspace=0.5)
plt.tight_layout()
plt.show()

Here mainly we need to focus on both 🟠 `orange` and 🟢 `green` bar.
In this graph we try to find relationship between numerical columns and their effect on having stroke. here we learn that

Features that `have less impact` on our target label are:
1. **body_mass_index**

Features which `have significant impact` on our target label are:
1. **blood_sugar_level**
2. **age**

In [ ]:
column_names = ['sex', 'high_blood_pressure', 'cardiovascular_disease', 'marrital_status', 'profession', 'stay']

plt.figure(figsize=(25, 18), dpi=400)

for index, columns in enumerate(column_names):
    plt.subplot(2, 3, index+1)
    sns.countplot(y=columns, data=dataset, palette='Set2', hue='stroke')
    plt.legend(labels=['No Stroke', 'Stroke'])
    plt.title(f'Stroke Count by {columns}', fontweight='bold', pad=20)
    plt.ylabel(columns, fontweight='bold')
    plt.xlabel('Count', fontweight='bold')
    # plt.gca().set_yticklabels(dataset['work_type'], rotation=45)
    plt.yticks(rotation=45)


plt.subplots_adjust(hspace=0.3)
plt.subplots_adjust(left=0.1, right=1, top=0.6, bottom=0.1, wspace=0.2)

plt.show()


In [ ]:
    # plt.subplot(2, 3, index+1)
columns = 'tobaco_use'
sns.countplot(y=columns, data=dataset, palette='Set2', hue='stroke')
plt.legend(labels=['No Stroke', 'Stroke'])
plt.title(f'Stroke Count by {columns}', fontweight='bold', pad=20)
plt.ylabel(columns, fontweight='bold')
plt.xlabel('Count', fontweight='bold')
# plt.gca().set_yticklabels(dataset['profession'], rotation=45)
plt.yticks(rotation=45)

Here mainly we need to focus on the 🟠 `orange` bar.
In this graph we try to find relationship between non numerical columns and their effect on having stroke. here we learn that

Values look fine here, there are no unwanted/error values.

Features that `have less impact` on our target label are:
1. **sex**
2. **Resident_type**

Features which `have significant impact` on our target label are:
1. **high_blood_pressure**
2. **heart_desease**
3. **marrital_status**
4. **profession**

🤔 Let's confirm this using heatmap of correlation metrix

In [ ]:
df_correlation = dataset[['body_mass_index', 'blood_sugar_level', 'age','stroke']]
df_cort = df_correlation.corr()
sns.heatmap(
    df_cort,   annot=True)

Here if you check against `stroke` and our three numerical columns you can see that value for `age:0.25` and `average_glucose_level:0.13` are more closer to 1 but `body_mass_index:0.042` is significantlly lower. Which confirms our earlier finding.

<a id='thoughts'></a>
## 💭 <span style="color: #20479b; font-weight: bold;">My thoughts on the dataset</span>

As per my initial Data Exploration,
- I can see that the dataset big enough to build a ML model.
- This dataset consits of null values in   `body_mass_index` column. Which I can delete it as it negligible rows of data and the body mass index cannot be `null`.
- I will be droping column `identity` as we do not need identity column while building a prediction model as will bring the accuracy down.
- We do have very less data for people who got stroke than in comparison to people to did not. Which makes it little imbalanced, so I will oversample the data.
- columns `body_mass_index`, `sex` and `resident_type` have less impact on the our target label
- Having said that the still have slight impact. Hence I will not delete them.
- Other than that the data looks pretty clean and I may not have to do a lot of data cleaning.
- As I will have to train the model and then later test it, I will be splitting our dataset into two parts. Training and Testing split.


Evaluation Metrics:
For this dataset I will be using, area under the curve, Accuracy, precision, F1 are the evaluation metrics.



Considering all these above points, I will be following the below steps to 

**⚙️ pre-process**
1. ❎ Drop all the rows containing `null` values
2. ❎ Drop columns `identity`
3. ➗ Spliting our main datset into two parts, one part for training and another for testing dataframe
    - Doing it before testing or 
    - to avoid data Scalling and oversampling.

**🔧 feature Engineer**
1. ⚖️ Scalling the numerical and categorical labels

2. 📈 Oversampling the data for stroke = 1, Because of two main reasons.
    - Our focus is on stroke detection and data for people who have stroke is very less.
    - We do not have to worry about data size as well here, since it's not very huge.

<a id='processing' ></a>
## ⚙️ <span style="color: #20479b; font-weight: bold;">Pre-processing the data</span>


1. ❎ Drop all the rows containing `null` values.

In [ ]:
dataset = dataset.dropna()

In [ ]:
print("Number of rows.   :", len(dataset))
print("="*30)

dataset.isnull().sum()

2. ❎ Drop columns `identity`.

In [ ]:

dataset = dataset.iloc[:,[1,2,3,4,5,6,7,8,9,10,11]]
dataset.sample(1)

3. ➗ Spliting our main datset into two parts, one part for training and another for testing dataframe

In [ ]:
dataset.iloc[:,[0,1,2,3,4,5,6,7,8,9]]
# y = dataset.iloc[:,[11]]

In [ ]:
x = dataset.iloc[:,[0,1,2,3,4,5,6,7,8,9]]
y = dataset.iloc[:,[10]]

x_train_df,  x_test_df,   y_train_df,  y_test_df = sklearn.model_selection.train_test_split(  x , y  ,  test_size=0.2,  shuffle=True,random_state  =  40)

print("dataset size      :",dataset.shape)
print("x_train_df train size:",x_train_df.shape)
print("x_test_df test size :",x_test_df.shape)

<a id='feature' ></a>
## 🔧 <span style="color: #20479b; font-weight: bold;">Feature Engineering</span>


1. ⚖️ Scalling the numerical and categorical labels

In [ ]:
col_num = [ 'age', 
            'high_blood_pressure', 
            'cardiovascular_disease', 
            'blood_sugar_level', 
            'body_mass_index'
            ]
col_cat = ['sex', 
            'marrital_status',  
            'profession',  
            'stay',  
            'tobaco_use'
            ]

std_scaler = sklearn.preprocessing.StandardScaler()
one_hot_enc = sklearn.preprocessing.OneHotEncoder(sparse_output=False, dtype=np.int64)

ct = sklearn.compose.ColumnTransformer([('std_scaler', std_scaler,col_num),('one_hot_enc', one_hot_enc,col_cat)],remainder='passthrough')
# ,('one_hot_enc', one_hot_enc,col_cat)]
ct.fit(   x_train_df )
ct.set_output( transform="pandas"  )
x_train_df = ct.transform(  x_train_df )
x_test_df = ct.transform(   x_test_df )
# x_train_df.head(5)

x_test_df.sample(1)

2. 📈 Oversampling the data

In [ ]:
mod_smote =  SMOTE(  )
x_train_smot, y_train_smot   =    mod_smote.fit_resample(
                                                            x_train_df, 
                                                            y_train_df)

print("x_train_smot size :",x_train_smot.shape)
print("y_train_smot size :",y_train_smot.shape)

In [ ]:
y_train_smot = y_train_smot.squeeze()
y_train_df

In [ ]:
y_train_smot.value_counts().plot(kind='pie',autopct='%1.0f%%')
plt.title('Proportion for stroke column', fontweight='bold')

It is now balanced as it's added synthetic data.

We can see that it had added more synthetic data to our dataset, which is good for us.

<a id='selection'></a>
## 🗂️ <span style="color: #20479b; font-weight: bold;">Model Selection</span>

So I will be trying 5  models
1. Linear Regression
2. Support Vector Classifier
3. Random Forest Classifier
4. Xgboost 
5. KNearestNeighbour

I am fitting the model for `x_train_smot` and `y_train_smot`

In [ ]:

log_reg = sklearn.linear_model.LogisticRegression(max_iter=1200)
svc = sklearn.svm.SVC(  )
ran_for_cls =  sklearn.ensemble.RandomForestClassifier(  random_state = 1200)
xg_bst = XGBClassifier(  eval_metric = 'error'  )
k_n_n =   sklearn.neighbors.KNeighborsClassifier(    )


log_reg.fit(x_train_smot,
            y_train_smot)
svc.fit(x_train_smot,
            y_train_smot)
ran_for_cls.fit(x_train_smot,
            y_train_smot)
xg_bst.fit(x_train_smot,
            y_train_smot)
k_n_n.fit(x_train_smot.values,
            y_train_smot.values)

In [ ]:
type(x_test_df)

<a id='assess'></a>
## ✅ <span style="color: #20479b; font-weight: bold;">Model Assesment</span>

In [ ]:
# ================Logistic Regression==================
val_met_list = []

y_pred_log_reg = log_reg.predict(
                                 x_test_df
                                 )
val_met_list.append(['Logistic Regression',
                      cross_val_score(estimator=log_reg, X=x_train_smot, y=y_train_smot, cv=15).mean(), 
                      accuracy_score(y_test_df, y_pred_log_reg),
                      roc_auc_score(y_test_df, y_pred_log_reg),
                      precision_score(y_test_df, y_pred_log_reg),
                      f1_score(y_test_df, y_pred_log_reg)])

# =============Support Vector Classifier===============

y_pred_svc = svc.predict(
                         x_test_df
                         )
val_met_list.append(['Support Vector Classifier',
                      cross_val_score(estimator=svc, X=x_train_smot, y=y_train_smot, cv=15).mean(), 
                      accuracy_score(y_test_df, y_pred_svc),
                      roc_auc_score(y_test_df, y_pred_svc),
                      precision_score(y_test_df, y_pred_svc),
                      f1_score(y_test_df, y_pred_svc)])

# ================Random Forest========================

y_pred_ran_for_cls = ran_for_cls.predict(
                                         x_test_df
                                         )
val_met_list.append(['Random Forest',
                      cross_val_score(estimator=ran_for_cls, X=x_train_smot, y=y_train_smot, cv=15).mean(), 
                      accuracy_score(y_test_df, y_pred_ran_for_cls),
                      roc_auc_score(y_test_df, y_pred_ran_for_cls),
                      precision_score(y_test_df, y_pred_ran_for_cls),
                      f1_score(y_test_df, y_pred_ran_for_cls)])


# =======================Xgboost=========================

y_pred_xb = xg_bst.predict(
                         x_test_df
                         )
val_met_list.append(['Xgboost    ',
                      cross_val_score(estimator=xg_bst, X=x_train_smot, y=y_train_smot, cv=15).mean(), 
                      accuracy_score(y_test_df, y_pred_xb),
                      roc_auc_score(y_test_df, y_pred_xb),
                      precision_score(y_test_df, y_pred_xb),
                      f1_score(y_test_df, y_pred_xb)])

# ==========================KNN==============================

y_pred_knn = k_n_n.predict(
                         x_test_df.values
                         )
val_met_list.append(['KNN   ',
                      cross_val_score(estimator=k_n_n, X=x_train_smot.values, y=y_train_smot.values, cv=15).mean(), 
                      accuracy_score(y_test_df, y_pred_knn),
                      roc_auc_score(y_test_df, y_pred_knn),
                      precision_score(y_test_df, y_pred_knn),
                      f1_score(y_test_df, y_pred_knn)])


val_met_df = pd.DataFrame(val_met_list, 
                          columns=['Model Name','Avg ACC','ACC','AUC','PRE','F1'])
val_met_df = val_met_df.sort_values(
                                     ascending=False, by=['Avg ACC','ACC','AUC']
                                    )
val_met_df

In [ ]:
mdl_list = [log_reg,svc,ran_for_cls,xg_bst,k_n_n]
for i, mdl in enumerate(mdl_list):
    print(i)
    print(mdl)
    print(mdl_list[i])

In [ ]:
val_met_list = []


log_reg = sklearn.linear_model.LogisticRegression(max_iter=1200)
svc = sklearn.svm.SVC(  )
ran_for_cls =  sklearn.ensemble.RandomForestClassifier(  random_state = 1200)
xg_bst = XGBClassifier(  eval_metric = 'error'  )
k_n_n =   sklearn.neighbors.KNeighborsClassifier(    )


log_reg.fit(x_train_smot,
            y_train_smot)
svc.fit(x_train_smot,
            y_train_smot)
ran_for_cls.fit(x_train_smot,
            y_train_smot)
xg_bst.fit(x_train_smot,
            y_train_smot)
k_n_n.fit(x_train_smot.values,
            y_train_smot.values)


mdl_list = [log_reg,svc,ran_for_cls,xg_bst,k_n_n]

for i, mdl in enumerate(mdl_list):


    if mdl==k_n_n:
        x_train_smot_val = x_train_smot.values
        y_train_smot_val = y_train_smot.values
        x_test_df_val    = x_test_df.values
    else:
        x_train_smot_val = x_train_smot
        y_train_smot_val = y_train_smot
        x_test_df_val    = x_test_df

    y_pred_df = log_reg.predict(
                                x_test_df_val
                                )

    val_met_list.append([ mdl_list[i],
                        cross_val_score(estimator=mdl, X=x_train_smot_val, y=y_train_smot_val, cv=15).mean(), 
                        accuracy_score(y_test_df, y_pred_df),
                        roc_auc_score(y_test_df, y_pred_df),
                        precision_score(y_test_df, y_pred_df),
                        f1_score(y_test_df, y_pred_df)])


val_met_df = pd.DataFrame(val_met_list, 
                        columns=['Model Name','Avg ACC','ACC','AUC','PRE','F1'])
val_met_df = val_met_df.sort_values(
                                    ascending=False, by=['Avg ACC','ACC','AUC']
                                    )
val_met_df

As per our valuation metrics the best model out of them are
1. Random Forest
2. Xgboost

Let us try to tune these two models and try to find we can get the model to perform even better score.

<a id='tuning'></a>
## 🎛️ <span style="color: #20479b; font-weight: bold;">Hyper-Parameter Tuning</span>

I will using gridsearchcv and passing it multiple values so that it can choose a best value for us.

####  <span style=" font-weight: bold;">Random Forest Classifier</span>

In [ ]:
params = {'criterion':['gini', 'log_loss'],'n_estimators':[60,80,100,120,140],  'random_state' : [1200,None]}
param_result_list = []
grid_ser_cv = GridSearchCV(ran_for_cls,params,cv=5)
grid_ser_cv.fit(x_train_smot,y_train_smot)
df_results = pd.DataFrame(grid_ser_cv.cv_results_)
param_result_list.append([grid_ser_cv.best_score_,grid_ser_cv.best_params_])

param_result_df = pd.DataFrame(param_result_list, columns=['best_score_','best_params_',])
param_result_df

####  <span style=" font-weight: bold;">Support Vector Classifier</span>

In [ ]:
params = {'booster':['gbtree', 'gblinear', 'dart'],'eta':[0.1,0.2,0.3,0.4]}
param_result_list = []
grid_ser_cv = GridSearchCV(xg_bst,params,cv=5)
grid_ser_cv.fit(x_train_smot,y_train_smot)
df_results = pd.DataFrame(grid_ser_cv.cv_results_)
param_result_list.append([grid_ser_cv.best_score_,grid_ser_cv.best_params_])

param_result_df = pd.DataFrame(param_result_list, columns=['best_score_','best_params_',])
param_result_df

When we look at the scores of `random_forect_classifier` and `svm` model, we can come to a conclusion that both are a great model for this dataset.  
Having said that `random_forect_classifier` model perfomed sligtly better. Hence I willl choose that as my final model with `{'criterion': 'gini', 'n_estimators': 140, 'random_state': None}` as the hyperparameters.

<a id='Final'></a>
## 🏁 <span style="color: #20479b; font-weight: bold;">Final Model</span>

In [ ]:
final_ran_for_cls =  sklearn.ensemble.RandomForestClassifier(criterion= 'gini', n_estimators= 140,  random_state = None)
final_ran_for_cls.fit(x_train_smot,
            y_train_smot
            )
y_pred_ran_for_cls = final_ran_for_cls.predict(
                                         x_test_df
                                         )
print('Logistic Regression')
print('='*53)
print('Average Accuray -->',cross_val_score(estimator=log_reg, X=x_train_smot, y=y_train_smot, cv=5).mean())
print('Accuracy --------->',accuracy_score(y_test_df, y_pred_ran_for_cls))
print('ROC_AUS ---------->',roc_auc_score(y_test_df, y_pred_ran_for_cls))
print('Precision -------->',precision_score(y_test_df, y_pred_ran_for_cls))
print('F1 --------------->',f1_score(y_test_df, y_pred_ran_for_cls))


<a id='conclusion'></a>
## 💡 <span style="color: #20479b; font-weight: bold;">Conclusion</span>

To summarize everythin we did
- we started with loading the dataset
- In our eda we noticed that
    - identity column was unnessary.
    - body_mass_index column had null values.
    - dataset was pretty imbalanced.
- cleaned the data, by deleting null rows, dropping identity column, and split the data to avoid data leakage.
- And then we scaled the numerical and categorical values.
- oversampled the data using smote.

I tried 5 different model. Out of which `random forest classifier` and `xgboost` model performed the best.  
I tuned tuned the hyper parameters using `gridsearchCV` and found that amond both the models, `random forest classifier` outperformed `xgboost` slightly.

As per the valuation metrics I  found `random forest classifier` with `{'C': 1.4, 'degree': 4, 'kernel': 'poly'}` parameters provided the best with pretty good acccuracy of 93%.

With this model in place, now my company can use this model to take inputs from the users and prompt them if there are chances of getting a stroke for them. This solves the problem that we stated before. This will significantly help the users in reducing the rist of strokes, and notify them to get medical help as soon as possible.

I felt that if the data has more parametrics we could have improved more. Also `body_mass_index` of a person does impact on the chances of getting an stroke, but in this dataset it did not show the signs of it. Hence it will be better if data collection is done even more accurately to improve the accuracy of the model.

